<a href="https://colab.research.google.com/github/robburns3/collab-playground/blob/main/KaparthyNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import math

In [26]:
class Value:
    """
    Stores a single scalar value (data) and its computed gradient.

    Value can have a label for debugging/tracing and additional values
    for constructing a graph.
    """

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0
        self.label = label
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        """Value operation is applied as addition."""
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        """Value operation is applied as multiplication."""
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        """Value operation is applied as power."""
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        """
        Value operation is applied as rectified linear unit.
        ReLU(x) = max(0, x)
        """
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def tanh(self):
      """
      Value operation is applied as hyperbolic tangent.
      tanh(x) = (e^x - e^-x) / (e^x + e^-x)

      This is similar to ReLU in purpose but has different properties.
      """
      out = Value(math.tanh(self.data), (self,), 'tanh')

      def _backward():
        self.grad += (1 - out.data**2) * out.grad
      out._backward = _backward

      return out

    def backward(self):
        """Build a topological graph of the nodes and their dependencies."""
        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

a = Value(2.0, label="a")
b = Value(-3.0, label="b")
c = Value(10.0, label="c")
e = a * b; e.label = "e"
d = e + c; d.label = "d"
f = Value(-2.0, label="f")
L = d * f; L.label = "L"

L.backward()
print(a.grad)

6.0


In [27]:
import torch
import random

In [28]:
x1 = torch.Tensor([2.0]).double()
x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()
x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()
w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()
w2.requires_grad = True
b = torch.Tensor([6.881373]).double()
b.requires_grad = True

n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()




0.7071064519863152


In [29]:
class Neuron:
  """Holds a vector of output weights and a bias"""
  def __init__(self, nin):
    # Random initial weights
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    # Random initial bias
    self.b = Value(random.uniform(-1,1))

  def __call__(self, x):
    """Activation: Take input x and apply weights and bias"""
    # Activation function: w * x + b
    act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out

  def parameters(self):
    return self.w + [self.b]

x = [2.0, 3.0]
n = Neuron(2)
n(x)


Value(data=0.9129434914701305, grad=0)

In [30]:
class Layer:
  """Holds multiple neurons"""
  def __init__(self, nin, nout):
    # Neurons each with n inputs X number of total neurons
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

x = [2.0, 3.0]
L = Layer(2, 3)
L(x)

[Value(data=0.33751459994410926, grad=0),
 Value(data=0.25737214784757745, grad=0),
 Value(data=-0.9989260349808334, grad=0)]

In [44]:
class MultiLayerPerceptron:
  def __init__(self, nin, nouts):
    # Input layer size + hidden/output layer sizes
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]


MLP = MultiLayerPerceptron

x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

#n.parameters()

Value(data=-0.38770384784710904, grad=0)

In [45]:
# Intialize a new MLP
n = MLP(3, [4, 4, 1])

# Reset the neural net input/output values
# inputs, or X
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
# desired outputs, or Y
ys = [1.0, -1.0, -1.0, 1.0]

ypred = [n(x) for x in xs]
ypred

[Value(data=0.9388501284927419, grad=0),
 Value(data=0.8415685452608842, grad=0),
 Value(data=0.8911804778465063, grad=0),
 Value(data=0.9271875882392496, grad=0)]

In [46]:
num_steps = 20
nudge = 0.05
for k in range(num_steps):
    # Forward pass
    ypred = [n(x) for x in xs]
    loss = sum((ypred -ytruth)**2 for ytruth, ypred in zip(ys, ypred))

    # Backward pass
    # NOTE: Common mistake is not clear the gradiant value after each backprop
    # Need to call zero_grad() basically
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    # Apply gradient descent
    for p in n.parameters():
        # Note that the gradiant points in the direction of the loss, so we want
        # to go in the opposite direction to reduce the loss
        p.data += -nudge * p.grad

    print(k, loss.data)


0 6.9769792607738
1 5.907351373901376
2 3.187086792952698
3 0.9430072982145934
4 0.5078416587183198
5 0.3294704302244125
6 0.23728515052628069
7 0.18223434652380324
8 0.14621707678882323
9 0.12110072102131447
10 0.10274021774923435
11 0.08882097382665159
12 0.07795852306066971
13 0.06927895008234383
14 0.0622061213235733
15 0.05634633908657351
16 0.05142232458240943
17 0.04723369118808157
18 0.0436323576317425
19 0.040506744535705214


In [47]:
# Show the predictions
# How do they compare with the inputs? Closer, but could be better
ypred

[Value(data=0.9702052696814796, grad=-0.05958946063704085),
 Value(data=-0.9304388286621837, grad=0.1391223426756325),
 Value(data=-0.8510347360058204, grad=0.2979305279883593),
 Value(data=0.8877965591160093, grad=-0.2244068817679814)]